In [65]:
import geopandas as gpd
import pandas as pd
import gpxpy
import os

# Práca
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"

# Load the CSV data into a DataFrame
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

# Replace the file path with your actual file path
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame

In [66]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

In [67]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - TT') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - SE') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - DS')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2024"] != 0]

# filtered_bridges = filtered_bridges[filtered_bridges["Material"]
#                                     == "prefabrikovaný predpätý betón"]

# filtered_bridges = filtered_bridges[
#     (filtered_bridges["n_2024"] == 4) |
#     (filtered_bridges["n_2024"] == 5)
# ]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(382, 44)

In [68]:
import folium


def plot_filtered_bridges(df, gdf, regions):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng.svg"

    for index, row in df.iterrows():
        ic_size = 25
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{
            row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{
            row['Trieda_PK']}</td></tr>"

        popup_html += "<tr><th style='text-align:left;'>Dĺžka premostenia:</th><td>"
        if pd.notna(row['Dĺžka_premostenia']):
            popup_html += f"{float(row['Dĺžka_premostenia'])} m"
        else:
            popup_html += "Unknown"
        popup_html += "</td></tr>"

        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{
            row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{
            row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{
            row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({
            row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {
            row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{
            row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

    # Filter the GeoDataFrame to include only the specified regions
    region_gdf = gdf[gdf['NM4'].isin(regions)]

    # Define custom style for the regions' polygons
    def style_function(x): return {
        'fillColor': '#FF6600', 'color': '#FF6600', 'weight': 2}

    # Add the boundaries of the specified regions to the map with the custom style
    folium.GeoJson(region_gdf, name='Regions Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame and adjust regions as needed)
regions_to_plot = ['Trnavský']
map = plot_filtered_bridges(filtered_bridges, gdf, regions_to_plot)
map.save('Bridges_TA.html')
map

In [69]:
import plotly.graph_objects as go

# Creating the histogram for filtered_bridges
trace_filtered = go.Histogram(
    x=filtered_bridges["n_2024"],
    name='Mosty SÚC TTSK',
    marker_color='#e7a564 ',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=1  # Set offsetgroup to 1 for the bars to be side by side
)

# Creating the histogram for df_bridges_all
trace_all = go.Histogram(
    x=df_bridges_all["n_2024"],
    name='Všetky mosty',
    marker_color='#46a7a2',
    opacity=0.75,
    histnorm='percent',
    offsetgroup=2  # Set offsetgroup to 2 for the bars to be side by side
)

# Creating the figure and adding the histograms
fig = go.Figure(data=[trace_filtered, trace_all])

# Update the layout with Montserrat font
fig.update_layout(
    title_text='Relatívne zastúpenie STS mostov',
    xaxis_title_text='stavebno-technický stav',
    yaxis_title_text='zastúpenie (%)',
    barmode='group',  # Set barmode to "group" for bars to be side by side
    font=dict(
        family="Montserrat, sans-serif",  # Set font family to Montserrat
        size=14
    ),
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=1.10,  # Position the legend slightly above the plot
        xanchor="left",  # Anchor legend to the left
        x=.615  # Align legend to the left
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=50),  # Adjust margins for tight layout
    bargap=0.2
)

# Adjusting the opacity for better visualization
fig.update_traces(opacity=0.75)

fig.show()

In [70]:
import pandas as pd

# Initialize an empty DataFrame for relative condition counts across years
relative_condition_counts = pd.DataFrame()

for year in range(2012, 2025):
    column_name = f"n_{year}"
    # Exclude bridges not assessed in the year (condition = 0)
    year_data = filtered_bridges[filtered_bridges[column_name] > 0]  # NR
    # year_data = df_bridges_all[df_bridges_all[column_name] > 0] # Všetky mosty
    # Count occurrences of each condition for the year
    counts = year_data[column_name].value_counts().rename_axis(
        'Condition').reset_index(name=column_name)
    # Calculate relative counts (percentage) for the year
    total = counts[column_name].sum()
    counts[column_name] = (counts[column_name] / total) * 100

    if relative_condition_counts.empty:
        relative_condition_counts = counts
    else:
        relative_condition_counts = pd.merge(
            relative_condition_counts, counts, on='Condition', how='outer')

# Fill NaN values with 0 for years when a condition was not present
relative_condition_counts.fillna(0, inplace=True)
# Sort the DataFrame by 'Condition' for a consistent order
relative_condition_counts.sort_values('Condition', inplace=True)

In [71]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#46a7a2', '#e7a564',
                 '#8bae7c', '#353a29', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Transpose the DataFrame for plotting: years as columns, conditions as rows
data_for_plotting = relative_condition_counts.set_index(
    'Condition').T.reset_index()
data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)

# Update year labels
years = ['{}'.format(year) for year in range(2012, 2025)]
data_for_plotting['Year'] = years

# Create a figure
fig = go.Figure()

# Add a trace for each condition in reverse order
for i, condition in reversed(list(enumerate(relative_condition_counts['Condition'].unique()))):
    fig.add_trace(go.Scatter(
        x=data_for_plotting['Year'],
        y=data_for_plotting[condition],
        mode='lines',
        stackgroup='one',  # This option stacks the areas
        name=str(condition),  # Label for the legend
        # Apply color from the palette
        line=dict(color=color_palette[i % len(color_palette)])
    ))

# Update the layout
fig.update_layout(
    title='Vývoj STS mostov SÚC TTSK v čase',
    # xaxis_title='rok',
    yaxis_title='',
    yaxis=dict(ticksuffix="%", range=[0, 100]
               ),  # Set y-axis range from 0 to 100%
    # Set font family to Montserrat
    font=dict(family="Montserrat, sans-serif", size=14),
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=1.06,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=1  # Align legend to the right
    ),
    # Set x-axis tick values and labels
    xaxis=dict(
        tickvals=[str(year) for year in range(2012, 2025)],
        ticktext=['2012', '2013', '2014', '2015', '2016', '2017',
                  '2018', '2019', '2020', '2021', '2022', '2023', "2024"]
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=25),  # Adjust margins for tight layout
    width=500,  # Set figure width
    height=500  # Set figure height
)

# Show the figure
fig.show()

In [72]:
import pandas as pd

# Initialize an empty DataFrame for relative condition counts across years
relative_condition_counts = pd.DataFrame()

for year in range(2012, 2025):
    column_name = f"n_{year}"
    # Exclude bridges not assessed in the year (condition = 0)
    year_data = df_bridges_all[df_bridges_all[column_name] > 0]  # NR
    # year_data = df_bridges_all[df_bridges_all[column_name] > 0] # Všetky mosty
    # Count occurrences of each condition for the year
    counts = year_data[column_name].value_counts().rename_axis(
        'Condition').reset_index(name=column_name)
    # Calculate relative counts (percentage) for the year
    total = counts[column_name].sum()
    counts[column_name] = (counts[column_name] / total) * 100

    if relative_condition_counts.empty:
        relative_condition_counts = counts
    else:
        relative_condition_counts = pd.merge(
            relative_condition_counts, counts, on='Condition', how='outer')

# Fill NaN values with 0 for years when a condition was not present
relative_condition_counts.fillna(0, inplace=True)
# Sort the DataFrame by 'Condition' for a consistent order
relative_condition_counts.sort_values('Condition', inplace=True)

In [73]:
import plotly.graph_objects as go

# Define the color palette
color_palette = ['#46a7a2', '#e7a564',
                 '#8bae7c', '#353a29', '#eae1a9',
                 '#c25469', '#2d505d'
                 ]

# Transpose the DataFrame for plotting: years as columns, conditions as rows
data_for_plotting = relative_condition_counts.set_index(
    'Condition').T.reset_index()
data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)

# Update year labels
years = ['{}'.format(year) for year in range(2012, 2025)]
data_for_plotting['Year'] = years

# Create a figure
fig = go.Figure()

# Add a trace for each condition in reverse order
for i, condition in reversed(list(enumerate(relative_condition_counts['Condition'].unique()))):
    fig.add_trace(go.Scatter(
        x=data_for_plotting['Year'],
        y=data_for_plotting[condition],
        mode='lines',
        stackgroup='one',  # This option stacks the areas
        name=str(condition),  # Label for the legend
        # Apply color from the palette
        line=dict(color=color_palette[i % len(color_palette)])
    ))

# Update the layout
fig.update_layout(
    title='Vývoj STS mostov v čase',
    # xaxis_title='rok',
    yaxis_title='percentuálne zastúpenie',
    yaxis=dict(ticksuffix="%", range=[0, 100]
               ),  # Set y-axis range from 0 to 100%
    # Set font family to Montserrat
    font=dict(family="Montserrat, sans-serif", size=14),
    legend=dict(
        orientation="h",  # Horizontal legend orientation
        yanchor="top",  # Anchor legend to the top
        y=1.06,  # Position the legend slightly above the plot
        xanchor="right",  # Anchor legend to the right
        x=1  # Align legend to the right
    ),
    # Set x-axis tick values and labels
    xaxis=dict(
        tickvals=[str(year) for year in range(2012, 2025)],
        ticktext=['2012', '2013', '2014', '2015', '2016', '2017',
                  '2018', '2019', '2020', '2021', '2022', '2023', "2024"]
    ),
    plot_bgcolor='#EEEEEE',
    margin=dict(l=0, r=20, t=50, b=25),  # Adjust margins for tight layout
    width=500,  # Set figure width
    height=500  # Set figure height
)

# Show the figure
fig.show()